## Sproj Board Scheduler v2

A script to read faculty and students availability for the boards week, and schedule all boards, accounting for their target composition, and student availability.

Some of the data used by this script is sensitive, and had to be stored outside of git.

**v2** uses pre-parsed data from WhenIsGood.

In [1]:
import pandas as pd
import time
import datetime
import copy
import random
from IPython.display import HTML, display

In [2]:
hideNames = 1 # set 0 for troubleshooting, set 1 before githubbing

In [3]:
composition = pd.read_csv('../../sensitive_data/boards.csv')
can_do = pd.read_csv('../../sensitive_data/availability.csv')

In [4]:
def create_grid(firstday, shifts, hou_min=8, hou_max=21):
    """Creates time grid (day#, time tuples) and good-sounding day names."""
    first_date = datetime.datetime.strptime(firstday, "%b %d %Y")
    grid = [] # Array of tuples: day (as a #), hour (as a 24h int number)
    day_name = {}
    for i in range(len(shifts)):
        s = datetime.datetime.strftime(first_date+datetime.timedelta(days=shifts[i]),"%a, %b %d")
        #print(s)
        day_name[i] = s
        
        # Annoying facts: 
        # 1) both time and datetime have strftime() method, but the syntax is different (sequence of arguments)
        # 2) in datetime, most useful stuff sits in datetime.datetime, but not all (timedelta doesn't)
        # 3) although both time and datetime have strptime, only one (datetime) works with datetime.timedelta
        
        for t in range(hou_min, hou_max):
            grid.append((i,t))

    return grid, day_name

# Test
grid, day_name = create_grid('May 04 2020', [0,1,2,3,4,5,7,8,9,10,11,12])
print(day_name)

{0: 'Mon, May 04', 1: 'Tue, May 05', 2: 'Wed, May 06', 3: 'Thu, May 07', 4: 'Fri, May 08', 5: 'Sat, May 09', 6: 'Mon, May 11', 7: 'Tue, May 12', 8: 'Wed, May 13', 9: 'Thu, May 14', 10: 'Fri, May 15', 11: 'Sat, May 16'}


In [5]:
class Faculty:
    """One faculty member, and their availability."""
    def __init__(self,name):
        self.name = name      # To be printed
        self.id = name        # to be used as id
        self.avail = []
        
    def __str__(self):
        #return str(self.name) + str(self.avail)
        return "%12s \t" % (self.name) + ''.join([f"{i:d}" for i in self.avail])
    
    def initAvail(self, grid, table):
        self.avail = [1]*len(grid)
        for i in range(len(grid)):
            check = table.loc[(table['Name']==self.id) & 
                              (table['Day']==grid[i][0]) &
                              (table['Time']==grid[i][1]), 'Cando']
            if len(check)==0:
                self.avail[i] = 1 # Set unknown ones to always available
            else:
                self.avail[i] = check.values[0]
            
    def updateAvail(self,grid,g,newVal=0):
        self.avail = [self.avail[i] if grid[i]!=g else newVal for i in range(len(self.avail))]
        
    def book(self,ig):
        self.avail[ig] = 0
        
        
# Populate faculty availability
faculty_list = list(composition['advisor'].append(composition['mem2']).append(composition['mem3']))
faculty_list = list(set([s.strip() for s in faculty_list])) # Remove trailing spaces, and get unique values
#print(faculty_list)

faculty_ids = {'Perron': 'Gabriel Perron', 'Robertson': 'Bruce Robertson', 'Keesing': 'Felicia', 
              'Jude': 'Brooke', 'Dueker': 'Eli Dueker', 'Collins': 'Cathy', 'Bennett': 'Heather',
              'Jain': 'Swapan Jain-CANNOT'}

faculty = []
for fn in faculty_list:
    f = Faculty(fn)
    if f.name in faculty_ids:
        f.id = faculty_ids[f.name]
    f.initAvail(grid, can_do)
    faculty.append(f)    

for f in faculty:
    print(f)

   Khakhalin 	111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
   Robertson 	111111111100010011111110001111011111000111000011100011001111110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
        Jain 	111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
     Bennett 	011100111000001110001100000111000010000011100011000001110000000000000000000000011100111000001110001100000111000010000011100011000001110000000000000000000000
      Perron 	111110000000011100000000000000001110000111110000000000000011100000000000000000110110000000011111000000000000001110000111110000000000000011100000000000000000
      Dueker 	11000000110011000000011001100000001100110000000110011000000011001100000001100110000000110011000000011001100000001100110000111110011

In [6]:
class Board:
    '''Board object (that also has all student information).'''
    
    def __init__(self, record): # Creator
        """Creates a student (board) record from a Pandas row-series.
        The row-series should come from the COMPOSITION dataframe."""
        
        self.student = record['first'] + ' ' + record['last']
        if hideNames:
            self.student = ''.join(random.sample(self.student.lower(),len(self.student)))
        self.id = self.student
        self.members = list(record[['advisor','mem2','mem3']])
        self.email = record['email']
        # self.type = record['type']
        self.avail = []  # Placeholder: availability grid
        self.time = []
        
    def __str__(self):
        s = "%20s" % (self.student) + '\t' 
        s += ' '.join([m[:4] for m in self.members]) + '\t' # Shortened version
        s += ''.join(['%d' % i for i in self.avail])
        return s
    
    def initTimes(self, grid, table):
        '''Sets boards initial options, based on students availability.
        While v1 used BIP data for this, this version uses whenisgood data.'''
        self.avail = [1]*len(grid)
        was_successful = 1
        for i in range(len(grid)):
            check = table.loc[(table['Name']==self.id) & 
                              (table['Day']==grid[i][0]) &
                              (table['Time']==grid[i][1]), 'Cando']
            if len(check)==0:
                was_successful = 0
                self.avail[i] = 1 # Set unknown ones to always available
            else:
                self.avail[i] = check.values[0]
        if not was_successful:
            print(f'Warning: missing data for {self.student}.')
                                
    def narrowBoardTime(self):
        '''Additional requirements on some types of boards'''
        #if self.type=="final": # For final boards, make Monday unavailable, as sprojes are due that day
        #    for ig in range(len(grid)):
        #        if grid[ig][0]=='m':
        #            self.avail[ig] = 0
        return
    
    def refreshFac(self,faculty):
        '''Filters boards based on current faculty availability'''
        for f in faculty:
            for facname in self.members:
                if f.name==facname:
                    self.avail = [self.avail[i]*f.avail[i] for i in range(len(self.avail))]
                    
# ----- Prepare id translation
# Translate from board names to self-reported names in the doodle
alt_names = {'Elizabeth Thomas':'Beth Thomas', 'Nadia Russell':'Nadia', 'Gabby Hartman':'Gabrielle Hartman',
            'Bruno DiNucci':'Bruno Di Nucci'}
                    
# ----- Populate the table
boards = []
print('Student availability:')
for i in range(len(composition)):
    b = Board(composition.loc[i,])
    if b.student in alt_names:
        b.id = alt_names[b.student]
    b.initTimes(grid, can_do)
    #print(b)
    boards.append(b)
    
for b in boards:
    b.narrowBoardTime()
    b.refreshFac(faculty)
    
# Rearrange from those that are harder to schedule to those that are easier
niceness = [sum(b.avail) for b in boards]
ind = [i for _,i in sorted(zip(niceness,range(len(niceness))))]
boards = [boards[i] for i in ind]

print('\nFull Board availability:')
for b in boards:
    print(b)

Student availability:

Full Board availability:
     tilnooyelastnpr	Duek Kees Perr	010000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000
       baaym rntghab	Robe Perr Duek	110000000000010000000000000000000010000100000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
      nhhnehcaia krr	Perr Duek Duek	110000000000010000000000000000000010000100000000000000000000100000000000000000100000000000010000000000000000000010000100000000000000000011100000000000000000
        yo nndaazrdm	Robe Duek Duek	110000001100010000000110001000000011000100000001100010000000110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
       licl ctyhcmae	Kees Robe Robe	011101100000000010100000000111001010000011000000000001000110000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [7]:
# Backup, just so that the cell below could be rerun without ruining the data
bb = copy.deepcopy(boards)
bf = copy.deepcopy(faculty)

In [8]:
boards = copy.deepcopy(bb) # Copy back from a backup, as the routine below changes the data
faculty = copy.deepcopy(bf)

for b in boards:
    b.refreshFac(faculty)
    temp = [i for i in range(len(b.avail)) if (b.avail[i]==1)]
    if len(temp)==0:
        print('Cannot solve the puzzle for this board:')
        print(b)
        break
    i_grid = min(temp)
    b.time = grid[i_grid]
    b.avail[i_grid] = 0
    for f in faculty:
        if f.name in b.members:
            f.book(i_grid)
    print('%s\t%s\t' % (day_name[grid[i_grid][0]],grid[i_grid][1]), end='')
    print(b)
    
#for b in boards:
#    print(b)

Mon, May 04	9	     tilnooyelastnpr	Duek Kees Perr	000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000
Mon, May 04	8	       baaym rntghab	Robe Perr Duek	000000000000010000000000000000000010000100000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Tue, May 05	8	      nhhnehcaia krr	Perr Duek Duek	000000000000000000000000000000000010000100000000000000000000100000000000000000100000000000010000000000000000000010000100000000000000000011100000000000000000
Mon, May 04	16	        yo nndaazrdm	Robe Duek Duek	000000000100000000000110001000000011000100000001100010000000110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Mon, May 04	10	       licl ctyhcmae	Kees Robe Robe	000101100000000010100000000111001010000011000000000001000110000000000000000000000000000000000000000000000000000000000000

In [9]:
# Make a summary table

def us_time(time):
    """Add a for am and p for pm"""
    if time>12:  return ("%dp" % (time-12))
    else:        return ("%da" % (time))

def printboard(b,mode="none"):
    """Print one board in one of three pre-defined formats."""
    if mode=="none":
        print("%22s\t%s\t%s\t" % (b.student,day_name[b.time[0]],us_time(b.time[1])),end='')
        for fn in set(b.members): # Set around it, to avoid double-output for 2-person boards
            print("%10s\t" % (fn),end='')
        print()
    elif mode=="tabs":
        if not hideNames:
            print("%s\t%s\t%s\t" % (b.student,day_name[b.time[0]],us_time(b.time[1])),end='')
        else:
            print("%s\t%s\t%s\t" % (b.student,day_name[b.time[0]],us_time(b.time[1])),end='')
        for fn in b.members:
            print("%s\t" % (fn),end='')
        print()
    elif mode=="html":
        s = ''
        s += "<tr><td>%s</td><td>%s %d</td><td>" % (b.student,day_name[b.time[0]],b.time[1])
        for fn in b.members:
            s += "%s " % (fn)
        s += "</td></tr>"
        display(HTML(s))
        
ind = [min([i for i in range(len(grid)) if grid[i]==b.time]) for b in boards] # All time slots that were taken
ind = [j for (i,j) in sorted(zip(ind,range(len(ind))))] # Now sorted by when they happen
boards = [boards[i] for i in ind]

count_faculty = {}
for b in boards:
    printboard(b,mode="none")
    for fn in set(b.members):
        count_faculty[fn] = count_faculty.get(fn,0)+1
        
print()
for key,val in count_faculty.items():
    print(key, ':', val)

         baaym rntghab	Mon, May 04	8a	    Dueker	 Robertson	    Perron	
          ngecl etenyd	Mon, May 04	8a	  Tibbetts	      Jude	
       tilnooyelastnpr	Mon, May 04	9a	    Dueker	   Keesing	    Perron	
           edinhfcda i	Mon, May 04	9a	 Khakhalin	   Bennett	
          lusinj bogia	Mon, May 04	9a	  Tibbetts	      Jude	
         licl ctyhcmae	Mon, May 04	10a	 Robertson	   Keesing	
         nuicodrnibc u	Mon, May 04	10a	   Bennett	    Perron	
          lmehieu chla	Mon, May 04	10a	  Tibbetts	      Jude	
         dlael siarusn	Mon, May 04	11a	   Keesing	    Perron	
       hmeceoar licrda	Mon, May 04	11a	 Khakhalin	   Bennett	
           vnacyoean m	Mon, May 04	12a	 Khakhalin	    Perron	
            nsaaai tqr	Mon, May 04	1p	 Khakhalin	  Tibbetts	
          yo nndaazrdm	Mon, May 04	4p	    Dueker	 Robertson	
        nhhnehcaia krr	Tue, May 05	8a	    Dueker	    Perron	
      epweejtesa nnaeu	Tue, May 05	9a	      Jain	    Perron	
   nt lmalmcasaeeraata	Tue, May 05	10a	  Tibbetts	    Per

In [10]:
#Sort by faculty:

for f in faculty:
    print()
    print(f.name)
    for b in boards:
        if f.name in b.members:
            printboard(b,mode="tabs")


Khakhalin
edinhfcda i	Mon, May 04	9a	Khakhalin	Bennett	Bennett	
hmeceoar licrda	Mon, May 04	11a	Khakhalin	Bennett	Bennett	
vnacyoean m	Mon, May 04	12a	Perron	Khakhalin	Khakhalin	
nsaaai tqr	Mon, May 04	1p	Khakhalin	Tibbetts	Tibbetts	

Robertson
baaym rntghab	Mon, May 04	8a	Robertson	Perron	Dueker	
licl ctyhcmae	Mon, May 04	10a	Keesing	Robertson	Robertson	
yo nndaazrdm	Mon, May 04	4p	Robertson	Dueker	Dueker	

Jain
epweejtesa nnaeu	Tue, May 05	9a	Perron	Jain	Jain	

Bennett
edinhfcda i	Mon, May 04	9a	Khakhalin	Bennett	Bennett	
nuicodrnibc u	Mon, May 04	10a	Bennett	Perron	Perron	
hmeceoar licrda	Mon, May 04	11a	Khakhalin	Bennett	Bennett	
emwtsh ielj	Tue, May 05	3p	Collins	Bennett	Bennett	

Perron
baaym rntghab	Mon, May 04	8a	Robertson	Perron	Dueker	
 tilnooyelastnpr	Mon, May 04	9a	Dueker	Keesing	Perron	
nuicodrnibc u	Mon, May 04	10a	Bennett	Perron	Perron	
dlael siarusn	Mon, May 04	11a	Keesing	Perron	Perron	
vnacyoean m	Mon, May 04	12a	Perron	Khakhalin	Khakhalin	
nhhnehcaia krr	Tue, May 05